In [81]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D
from tensorflow.keras.layers import Conv2DTranspose, LeakyReLU, Dropout, Embedding, Concatenate

In [82]:
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1]
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], 1))(li)
    # image input
    in_image = Input(shape=in_shape)
    # concat label as a channel
    merge = Concatenate()([in_image, li])
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output
    out_layer = Dense(1, activation='sigmoid')(fe)
    # define model
    model = Model([in_image, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [83]:
def define_generator(latent_dim, n_classes=10):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # linear multiplication
    n_nodes = 7 * 7
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((7, 7, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model

In [84]:
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [85]:
def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return [X, trainy]
 
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

In [86]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

In [87]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
            # generate 'fake' examples
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
            # prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    # save the generator model
    g_model.save('cgan_generator.h5')

In [88]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/468, d1=0.695, d2=0.694 g=0.693
>1, 2/468, d1=0.609, d2=0.699 g=0.688
>1, 3/468, d1=0.564, d2=0.708 g=0.678
>1, 4/468, d1=0.504, d2=0.725 g=0.664
>1, 5/468, d1=0.454, d2=0.745 g=0.647
>1, 6/468, d1=0.410, d2=0.778 g=0.624
>1, 7/468, d1=0.368, d2=0.811 g=0.602
>1, 8/468, d1=0.336, d2=0.848 g=0.597
>1, 9/468, d1=0.327, d2=0.848 g=0.611
>1, 10/468, d1=0.305, d2=0.794 g=0.661
>1, 11/468, d1=0.320, d2=0.746 g=0.732
>1, 12/468, d1=0.333, d2=0.640 g=0.850
>1, 13/468, d1=0.312, d2=0.549 g=0.980
>1, 14/468, d1=0.318, d2=0.474 g=1.088
>1, 15/468, d1=0.358, d2=0.411 g=1.167
>1, 16/468, d1=0.330, d2=0.427 g=1.162
>1, 17/468, d1=0.279, d2=0.469 g=1.049
>1, 18/468, d1=0.312, d2=0.556 g=0.891
>1, 19/468, d1=0.284, d2=0.652 g=0.773
>1, 20/468, d1=0.310, d2=0.760 g=0.676
>1, 21/468, d1=0.284, d2=0.878 g=0.603
>1, 22/468, d1=0.228, d2=0.946 g=0.562
>1, 23/468, d1=0.172, d2=0.997 g=0.550
>1, 24/468, d1=0.180, d2=1.004 g=0.567
>1, 25/468, d1=0.158, d2=0.949 g=0.634
>1, 26/468, d1=0.114, d2=0.768 g=0

>1, 209/468, d1=0.614, d2=0.633 g=0.837
>1, 210/468, d1=0.607, d2=0.652 g=0.819
>1, 211/468, d1=0.626, d2=0.683 g=0.790
>1, 212/468, d1=0.645, d2=0.682 g=0.791
>1, 213/468, d1=0.620, d2=0.674 g=0.793
>1, 214/468, d1=0.625, d2=0.695 g=0.775
>1, 215/468, d1=0.651, d2=0.701 g=0.746
>1, 216/468, d1=0.640, d2=0.753 g=0.736
>1, 217/468, d1=0.640, d2=0.735 g=0.749
>1, 218/468, d1=0.663, d2=0.739 g=0.717
>1, 219/468, d1=0.680, d2=0.739 g=0.712
>1, 220/468, d1=0.683, d2=0.763 g=0.706
>1, 221/468, d1=0.672, d2=0.780 g=0.712
>1, 222/468, d1=0.698, d2=0.762 g=0.725
>1, 223/468, d1=0.679, d2=0.753 g=0.734
>1, 224/468, d1=0.734, d2=0.755 g=0.725
>1, 225/468, d1=0.725, d2=0.756 g=0.732
>1, 226/468, d1=0.741, d2=0.714 g=0.755
>1, 227/468, d1=0.706, d2=0.701 g=0.740
>1, 228/468, d1=0.704, d2=0.707 g=0.775
>1, 229/468, d1=0.701, d2=0.690 g=0.771
>1, 230/468, d1=0.689, d2=0.673 g=0.789
>1, 231/468, d1=0.677, d2=0.676 g=0.780
>1, 232/468, d1=0.672, d2=0.677 g=0.776
>1, 233/468, d1=0.713, d2=0.663 g=0.800


>1, 414/468, d1=0.705, d2=0.645 g=0.801
>1, 415/468, d1=0.700, d2=0.676 g=0.763
>1, 416/468, d1=0.716, d2=0.684 g=0.760
>1, 417/468, d1=0.635, d2=0.671 g=0.751
>1, 418/468, d1=0.681, d2=0.677 g=0.743
>1, 419/468, d1=0.675, d2=0.701 g=0.736
>1, 420/468, d1=0.665, d2=0.689 g=0.730
>1, 421/468, d1=0.652, d2=0.696 g=0.734
>1, 422/468, d1=0.645, d2=0.702 g=0.736
>1, 423/468, d1=0.666, d2=0.720 g=0.737
>1, 424/468, d1=0.637, d2=0.675 g=0.762
>1, 425/468, d1=0.645, d2=0.659 g=0.760
>1, 426/468, d1=0.652, d2=0.663 g=0.787
>1, 427/468, d1=0.648, d2=0.647 g=0.789
>1, 428/468, d1=0.648, d2=0.655 g=0.799
>1, 429/468, d1=0.637, d2=0.640 g=0.802
>1, 430/468, d1=0.631, d2=0.625 g=0.793
>1, 431/468, d1=0.604, d2=0.610 g=0.808
>1, 432/468, d1=0.674, d2=0.663 g=0.788
>1, 433/468, d1=0.659, d2=0.688 g=0.768
>1, 434/468, d1=0.597, d2=0.735 g=0.725
>1, 435/468, d1=0.666, d2=0.722 g=0.717
>1, 436/468, d1=0.655, d2=0.760 g=0.667
>1, 437/468, d1=0.670, d2=0.782 g=0.689
>1, 438/468, d1=0.656, d2=0.778 g=0.721


>2, 154/468, d1=0.579, d2=0.647 g=0.842
>2, 155/468, d1=0.585, d2=0.612 g=0.865
>2, 156/468, d1=0.592, d2=0.608 g=0.856
>2, 157/468, d1=0.590, d2=0.613 g=0.854
>2, 158/468, d1=0.582, d2=0.599 g=0.884
>2, 159/468, d1=0.617, d2=0.606 g=0.860
>2, 160/468, d1=0.586, d2=0.615 g=0.803
>2, 161/468, d1=0.546, d2=0.640 g=0.861
>2, 162/468, d1=0.612, d2=0.631 g=0.803
>2, 163/468, d1=0.577, d2=0.647 g=0.766
>2, 164/468, d1=0.550, d2=0.709 g=0.757
>2, 165/468, d1=0.577, d2=0.696 g=0.755
>2, 166/468, d1=0.629, d2=0.738 g=0.765
>2, 167/468, d1=0.571, d2=0.693 g=0.737
>2, 168/468, d1=0.525, d2=0.687 g=0.749
>2, 169/468, d1=0.565, d2=0.657 g=0.783
>2, 170/468, d1=0.575, d2=0.652 g=0.820
>2, 171/468, d1=0.572, d2=0.639 g=0.826
>2, 172/468, d1=0.532, d2=0.649 g=0.822
>2, 173/468, d1=0.550, d2=0.620 g=0.876
>2, 174/468, d1=0.587, d2=0.626 g=0.849
>2, 175/468, d1=0.544, d2=0.655 g=0.825
>2, 176/468, d1=0.562, d2=0.644 g=0.834
>2, 177/468, d1=0.547, d2=0.648 g=0.812
>2, 178/468, d1=0.570, d2=0.696 g=0.792


>2, 359/468, d1=0.657, d2=0.637 g=0.810
>2, 360/468, d1=0.613, d2=0.699 g=0.749
>2, 361/468, d1=0.678, d2=0.788 g=0.723
>2, 362/468, d1=0.740, d2=0.755 g=0.700
>2, 363/468, d1=0.678, d2=0.742 g=0.732
>2, 364/468, d1=0.714, d2=0.709 g=0.807
>2, 365/468, d1=0.690, d2=0.634 g=0.852
>2, 366/468, d1=0.679, d2=0.585 g=0.929
>2, 367/468, d1=0.720, d2=0.549 g=0.952
>2, 368/468, d1=0.660, d2=0.536 g=0.957
>2, 369/468, d1=0.664, d2=0.556 g=0.899
>2, 370/468, d1=0.603, d2=0.597 g=0.878
>2, 371/468, d1=0.640, d2=0.658 g=0.798
>2, 372/468, d1=0.701, d2=0.772 g=0.704
>2, 373/468, d1=0.651, d2=0.803 g=0.693
>2, 374/468, d1=0.680, d2=0.765 g=0.740
>2, 375/468, d1=0.655, d2=0.722 g=0.786
>2, 376/468, d1=0.705, d2=0.643 g=0.867
>2, 377/468, d1=0.724, d2=0.569 g=0.911
>2, 378/468, d1=0.682, d2=0.554 g=0.991
>2, 379/468, d1=0.659, d2=0.601 g=0.883
>2, 380/468, d1=0.737, d2=0.605 g=0.865
>2, 381/468, d1=0.627, d2=0.638 g=0.788
>2, 382/468, d1=0.641, d2=0.702 g=0.735
>2, 383/468, d1=0.630, d2=0.770 g=0.737


...

>5, 195/468, d1=0.684, d2=0.646 g=0.781
>5, 196/468, d1=0.693, d2=0.649 g=0.787
>5, 197/468, d1=0.691, d2=0.650 g=0.791
>5, 198/468, d1=0.677, d2=0.667 g=0.748
>5, 199/468, d1=0.651, d2=0.666 g=0.781
>5, 200/468, d1=0.677, d2=0.670 g=0.777
>5, 201/468, d1=0.670, d2=0.653 g=0.770
>5, 202/468, d1=0.676, d2=0.686 g=0.782
>5, 203/468, d1=0.675, d2=0.670 g=0.767
>5, 204/468, d1=0.664, d2=0.682 g=0.770
>5, 205/468, d1=0.683, d2=0.637 g=0.790
>5, 206/468, d1=0.675, d2=0.653 g=0.773
>5, 207/468, d1=0.681, d2=0.651 g=0.797
>5, 208/468, d1=0.685, d2=0.656 g=0.783
>5, 209/468, d1=0.695, d2=0.642 g=0.785
>5, 210/468, d1=0.650, d2=0.632 g=0.777
>5, 211/468, d1=0.654, d2=0.646 g=0.792
>5, 212/468, d1=0.682, d2=0.659 g=0.768
>5, 213/468, d1=0.676, d2=0.659 g=0.750
>5, 214/468, d1=0.687, d2=0.675 g=0.751
>5, 215/468, d1=0.704, d2=0.650 g=0.758
>5, 216/468, d1=0.666, d2=0.639 g=0.760
>5, 217/468, d1=0.662, d2=0.678 g=0.775
>5, 218/468, d1=0.612, d2=0.684 g=0.760
>5, 219/468, d1=0.660, d2=0.667 g=0.759


>5, 400/468, d1=0.637, d2=0.669 g=0.756
>5, 401/468, d1=0.666, d2=0.682 g=0.758
>5, 402/468, d1=0.655, d2=0.703 g=0.743
>5, 403/468, d1=0.647, d2=0.690 g=0.761
>5, 404/468, d1=0.665, d2=0.652 g=0.789
>5, 405/468, d1=0.688, d2=0.640 g=0.795
>5, 406/468, d1=0.668, d2=0.656 g=0.778
>5, 407/468, d1=0.666, d2=0.646 g=0.824
>5, 408/468, d1=0.649, d2=0.660 g=0.792
>5, 409/468, d1=0.660, d2=0.674 g=0.770
>5, 410/468, d1=0.669, d2=0.651 g=0.776
>5, 411/468, d1=0.673, d2=0.673 g=0.792
>5, 412/468, d1=0.691, d2=0.648 g=0.785
>5, 413/468, d1=0.661, d2=0.661 g=0.771
>5, 414/468, d1=0.655, d2=0.658 g=0.815
>5, 415/468, d1=0.662, d2=0.652 g=0.780
>5, 416/468, d1=0.697, d2=0.626 g=0.792
>5, 417/468, d1=0.701, d2=0.651 g=0.789
>5, 418/468, d1=0.646, d2=0.667 g=0.764
>5, 419/468, d1=0.649, d2=0.689 g=0.765
>5, 420/468, d1=0.675, d2=0.666 g=0.771
>5, 421/468, d1=0.699, d2=0.669 g=0.783
>5, 422/468, d1=0.668, d2=0.649 g=0.779
>5, 423/468, d1=0.661, d2=0.658 g=0.784
>5, 424/468, d1=0.693, d2=0.630 g=0.795


>6, 140/468, d1=0.660, d2=0.686 g=0.771
>6, 141/468, d1=0.616, d2=0.680 g=0.809
>6, 142/468, d1=0.674, d2=0.627 g=0.813
>6, 143/468, d1=0.685, d2=0.639 g=0.819
>6, 144/468, d1=0.657, d2=0.651 g=0.805
>6, 145/468, d1=0.684, d2=0.654 g=0.783
>6, 146/468, d1=0.685, d2=0.673 g=0.780
>6, 147/468, d1=0.671, d2=0.662 g=0.764
>6, 148/468, d1=0.676, d2=0.651 g=0.774
>6, 149/468, d1=0.678, d2=0.652 g=0.770
>6, 150/468, d1=0.700, d2=0.657 g=0.788
>6, 151/468, d1=0.658, d2=0.652 g=0.793
>6, 152/468, d1=0.645, d2=0.678 g=0.782
>6, 153/468, d1=0.668, d2=0.672 g=0.772
>6, 154/468, d1=0.653, d2=0.704 g=0.751
>6, 155/468, d1=0.639, d2=0.691 g=0.758
>6, 156/468, d1=0.630, d2=0.668 g=0.758
>6, 157/468, d1=0.700, d2=0.669 g=0.793
>6, 158/468, d1=0.693, d2=0.618 g=0.815
>6, 159/468, d1=0.684, d2=0.624 g=0.819
>6, 160/468, d1=0.703, d2=0.626 g=0.809
>6, 161/468, d1=0.671, d2=0.653 g=0.766
>6, 162/468, d1=0.686, d2=0.667 g=0.768
>6, 163/468, d1=0.680, d2=0.681 g=0.751
>6, 164/468, d1=0.699, d2=0.680 g=0.762


>6, 345/468, d1=0.697, d2=0.637 g=0.802
>6, 346/468, d1=0.662, d2=0.639 g=0.792
>6, 347/468, d1=0.638, d2=0.653 g=0.793
>6, 348/468, d1=0.644, d2=0.622 g=0.803
>6, 349/468, d1=0.658, d2=0.646 g=0.817
>6, 350/468, d1=0.670, d2=0.735 g=0.742
>6, 351/468, d1=0.658, d2=0.709 g=0.767
>6, 352/468, d1=0.673, d2=0.707 g=0.737
>6, 353/468, d1=0.669, d2=0.678 g=0.773
>6, 354/468, d1=0.649, d2=0.669 g=0.820
>6, 355/468, d1=0.644, d2=0.607 g=0.849
>6, 356/468, d1=0.715, d2=0.629 g=0.836
>6, 357/468, d1=0.686, d2=0.634 g=0.822
>6, 358/468, d1=0.667, d2=0.649 g=0.795
>6, 359/468, d1=0.686, d2=0.655 g=0.767
>6, 360/468, d1=0.663, d2=0.677 g=0.744
>6, 361/468, d1=0.696, d2=0.678 g=0.748
>6, 362/468, d1=0.677, d2=0.655 g=0.784
>6, 363/468, d1=0.657, d2=0.644 g=0.789
>6, 364/468, d1=0.644, d2=0.636 g=0.856
>6, 365/468, d1=0.669, d2=0.639 g=0.788
>6, 366/468, d1=0.655, d2=0.653 g=0.822
>6, 367/468, d1=0.662, d2=0.665 g=0.769
>6, 368/468, d1=0.662, d2=0.672 g=0.759
>6, 369/468, d1=0.674, d2=0.651 g=0.784


KeyboardInterrupt: 